# Cost and Benefit Coastal Adaptation

Notebook environment to migrate netcdf files to zarr and geojson

In [1]:
# Use the black code formatter
%load_ext lab_black

### Configure OS independent paths

In [2]:
import os
import pathlib
import sys

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))


# Get root paths
home = pathlib.Path().home()
root = home.root

# Define both local and remote drives
local_data_dir = home.joinpath("ddata")
local_temp_dir = local_data_dir.joinpath("tmp")
p_dir = pathlib.Path(root, "p")
coclico_data_dir = p_dir.joinpath("11205479-coclico", "data")
coclico_cf_dir = coclico_data_dir.joinpath("CF")
ds_dirname = "06_adaptation_jrc"

# Project paths
local_auth_dir = local_data_dir.joinpath("AUTH_files")
remote_auth_dir = coclico_data_dir.joinpath("AUTH_files")
netcdf_dir = pathlib.Path("netcdf_files", "06.Coast and benefits of coastal adaptation")
json_dir = pathlib.Path("json_files", "06.Coast and benefits of coastal adaptation")

In [3]:
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

In [4]:
def get_fp(fn, suffix, remote_drive=True):
    file_dirs = {
        ".json": pathlib.Path(
            "json_files", "06.Coast and benefits of coastal adaptation"
        ),
        ".nc": pathlib.Path(
            "netcdf_files", "06.Coast and benefits of coastal adaptation"
        ),
    }
    local_auth_dir = local_data_dir.joinpath("tmp", "AUTH_files")
    remote_auth_dir = coclico_data_dir.joinpath("temp", "AUTH_files")

    if not remote_drive:
        return local_auth_dir.joinpath(file_dirs[suffix]).joinpath(fn + suffix)
    return remote_auth_dir.joinpath(file_dirs[suffix]).joinpath(fn + suffix)

In [5]:
fn_benefit = "benefitNoDiscounting"
fn_cost = "costNoDiscounting"
fn_cbr = "cbr"
fn_protection = "dZprotectionMean"

files = [fn_benefit, fn_cost, fn_cbr, fn_protection]

In [6]:
ds_benefit, ds_cost, ds_cbr, ds_protection = [
    xr.load_dataset(get_fp(fn, suffix=".nc", remote_drive=False)) for fn in files
]

In [7]:
df_benefit, df_cost, df_cbr, df_protection = [
    pd.read_json(get_fp(fn, suffix=".json", remote_drive=False)) for fn in files
]

### Load in raw data from p drive (excel sheets)

The nuts regions are not included as attributes in the netcdf files. The ones from the excel sheet are not present in recent nuts regsion shapefile by the EU. Therefore, project coordinates from data into current nuts regions. 

In [8]:
xlsx_benefit, xlsx_cost, xlsx_cbr, xlsx_protection = [
    pd.read_excel(local_temp_dir.joinpath("06_adaptation_jrc", f"{fn}.xlsx"))
    for fn in files
]

/home/calkoen/miniconda3/envs/coclico/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/calkoen/miniconda3/envs/coclico/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/calkoen/miniconda3/envs/coclico/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/calkoen/miniconda3/envs/coclico/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [9]:
from functools import reduce

xlsx_dfs = xlsx_benefit, xlsx_cost, xlsx_cbr, xlsx_protection
xlsx_merged = reduce(
    lambda l, r: pd.merge(l, r, on=["NUTS2 ID"], how="outer"), xlsx_dfs
)

/tmp/ipykernel_3388/4095977648.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Fossil Fuel Development_x', 'Sustainability_x'} in the result is deprecated and will raise a MergeError in a future version.
  lambda l, r: pd.merge(l, r, on=["NUTS2 ID"], how="outer"), xlsx_dfs


### Add nuts region

Nuts regions are obtained from eurostat, but the most recent nuts regions files do not
match the ones which are used in the datasets. The files describing the 2010 nuts regions
seem to match with the regions used in the studies. 

In [10]:
nuts_regions = gpd.read_file(
    local_data_dir.joinpath("tmp", "NUTS_RG_20M_2010_3857.shp")
)
nuts_regions = nuts_regions.to_crs("EPSG:4326")

In [11]:
# use one of the datasets to create a geodataframe
df_cost = df_cost.rename(
    {
        "latitude(degrees north of the NUTS2 regions centroid)": "latitude",
        "longitude(degrees east of the NUTS2 regions centroid)": "longitude",
    },
    axis="columns",
)

gdf_cost = gpd.GeoDataFrame(
    df_cost,
    geometry=gpd.points_from_xy(df_cost.longitude, df_cost.latitude),
    crs="EPSG:4326",
)

# Add nuts column from excel data
gdf_cost["NUTS_ID"] = xlsx_cost["NUTS2 ID"]

# inner join to keep only nuts regions used in dataset
nuts_regions = nuts_regions.merge(gdf_cost, on=["NUTS_ID"], how="inner")

# format dataframe
nuts_regions["instance"] = nuts_regions.index.values
nuts_regions = nuts_regions[
    ["instance", "NUTS_ID", "NAME_LATN", "CNTR_CODE", "geometry_x"]
]
nuts_regions = nuts_regions.rename(
    {
        "NUTS_ID": "acronym",
        "NAME_LATN": "name",
        "CNTR_CODE": "country",
        "geometry_x": "geometry",
    },
    axis="columns",
)
nuts_regions = gpd.GeoDataFrame(nuts_regions, crs="EPSG:4326")
nuts_regions.head()

# write result to geojson
# nuts_regions.to_file(
#     coclico_data_dir.joinpath("06_adaptation_jrc", "nuts_regions.geojson"),
#     driver="GeoJSON",
# )

,instance,acronym,name,country,geometry
0,0,BE23,Prov. Oost-Vlaanderen,BE,"POLYGON ((4.31117 51.12615, 4.17579 51.10121, ..."
1,1,BE25,Prov. West-Vlaanderen,BE,"POLYGON ((3.45973 50.76597, 3.45535 50.76456, ..."
2,2,BG33,Severoiztochen,BG,"POLYGON ((28.57888 43.73874, 28.60746 43.53937..."
3,3,BG34,Yugoiztochen,BG,"POLYGON ((26.98672 42.95232, 27.09443 42.95391..."
4,4,CY00,Kýpros,CY,"POLYGON ((33.00572 34.61297, 33.02382 34.58560..."


## Make datasets CF compliant 

In [12]:
# set lon/lat coordinates for each of the datasets
ds_benefit, ds_cost, ds_cbr, ds_protection = [
    ds.set_coords(["lon", "lat"]) for ds in [ds_benefit, ds_cost, ds_cbr, ds_protection]
]

In [14]:
# concat along a new layer dimension. The labels of this dimension are set as zero-terminated
# bytes
ds = xr.concat([ds_benefit, ds_cost, ds_cbr, ds_protection], dim="nlayers")
ds = ds.assign_coords(
    {
        "layers": (
            "nlayers",
            np.array(
                ["benefit", "cost", "cost_benefit_ratio", "protection"], dtype="S"
            ),
        )
    }
)

In [15]:
from shapely import wkb

# extract geometries of nut2 regions in well-known binary format
geoms = nuts_regions["geometry"].apply(lambda x: wkb.dumps(x))

# rename dims and add new data to dataset
ds = ds.rename({"row": "instance"})
ds = ds.assign_coords({"geometry": ("instance", geoms)})
# ds = ds.assign({"name": xr.Variable("instance", nuts_regions["name"].to_list())})

In [19]:
add_ds_attrs = {
    "geometry": {
        "long_name": "NUTS2 regions (polygons) in well-known binary format (wkb).",
        "geometry_type": "polygon",
        "units": "degree",
        "comment": "These NUTS2 regions (2010 version) are available at Eurostat.",
        "crs_wkt": f"{nuts_regions.crs.to_epsg()}",
    },
    "layers": {"long_name": "Socioeconomic layer."},
    # "name": {
    #     "long_name": "NUTS2 region names (2010 version).",
    #     "source": "NUTS2 regions (2010 version) are available at Eurostat.",
    # },
}

In [20]:
# global attrs
ds.attrs["Conventions"] = "CF-1.8"

# longitude attrs
ds["lon"].attrs["standard_name"] = "longitude"
ds["lon"].attrs["units"] = "degrees_east"
ds["lon"].attrs[
    "long_name"
] = "Longitude of the centroid of the NUTS2 region (2010 version)."
del ds["lon"].attrs["_CoordinateAxisType"]

# latitude attrs
ds["lat"].attrs["standard_name"] = "latitude"
ds["lat"].attrs["units"] = "degrees_north"
ds["lat"].attrs[
    "long_name"
] = "Latitude of the centroid of the NUTS2 region (2010 version)."
del ds["lat"].attrs["_CoordinateAxisType"]

# sustainability
ds["sustain"].attrs["long_name"] = "Sustainability"
ds["sustain"].attrs["units"] = "EUR 1 000 000"
del ds["sustain"].attrs["standard_name"]

# fossil fuel development
ds["ffd"].attrs["long_name"] = "Fossil fuel development"
ds["ffd"].attrs["units"] = "EUR 1 000 000"
del ds["ffd"].attrs["standard_name"]

KeyError: '_CoordinateAxisType'

In [21]:
for k, v in add_ds_attrs.items():
    ds[k].attrs = add_ds_attrs[k]

### Run CF checker

In [22]:
# save current dataset as netcdf in tmp directory
ds_outpath = local_temp_dir.joinpath("cbca_CF.nc")
ds.to_netcdf(path=ds_outpath)

In [32]:
ds_outpath

PosixPath('/home/calkoen/ddata/tmp/cbca_CF.nc')

In [23]:
# check using cfecker python library (default settings, hence, most current var, region, ..., etc. names)
from cfchecker.cfchecks import CFChecker

CFChecker().checker(str(ds_outpath))

CHECKING NetCDF FILE: /home/calkoen/ddata/tmp/cbca_CF.nc
Using CF Checker Version 4.1.0
Checking against CF Version CF-1.8
Using Standard Name Table Version 79 (2022-03-19T15:25:54Z)
Using Area Type Table Version 10 (23 June 2020)
Using Standardized Region Name Table Version 4 (18 December 2018)


------------------
Checking variable: lat
------------------

------------------
Checking variable: lon
------------------

------------------
Checking variable: sustain
------------------
ERROR: (3.1): Invalid units: EUR 1 000 000

------------------
Checking variable: ffd
------------------
ERROR: (3.1): Invalid units: EUR 1 000 000

------------------
Checking variable: layers
------------------

------------------
Checking variable: geometry
------------------
INFO: attribute geometry_type is being used in a non-standard way

ERRORS detected: 2
WARNINGS given: 0
INFORMATION messages: 1


{'global': {'FATAL': [],
  'ERROR': [],
  'WARN': [],
  'INFO': [],
  'VERSION': ['CHECKING NetCDF FILE: /home/calkoen/ddata/tmp/cbca_CF.nc',
   'Using CF Checker Version 4.1.0',
   'Checking against CF Version CF-1.8',
   'Using Standard Name Table Version 79 (2022-03-19T15:25:54Z)',
   'Using Area Type Table Version 10 (23 June 2020)',
   'Using Standardized Region Name Table Version 4 (18 December 2018)']},
 'variables': OrderedDict([('lat',
               {'FATAL': [],
                'ERROR': [],
                'WARN': [],
                'INFO': [],
                'VERSION': []}),
              ('lon',
               {'FATAL': [],
                'ERROR': [],
                'WARN': [],
                'INFO': [],
                'VERSION': []}),
              ('sustain',
               {'FATAL': [],
                'ERROR': ['(3.1): Invalid units: EUR 1 000 000'],
                'WARN': [],
                'INFO': [],
                'VERSION': []}),
              ('ffd',
   

### Write CF logs to p_drive as backlog

In [26]:
# define paths to save log files

cf_dir = coclico_cf_dir.joinpath(ds_dirname)
if not cf_dir.exists():
    cf_dir.mkdir()

In [28]:
from contextlib import redirect_stdout

# write CF logs to p_drive
with open(cf_dir.joinpath(ds_outpath.stem).with_suffix(".check"), "w") as f:
    with redirect_stdout(f):
        CFChecker().checker(str(ds_outpath))

### Copy files from local to p_drive

In [34]:
import shutil

# #TODO: fix permission error when copying to p_drive
# shutil.copy(ds_outpath, coclico_data_dir.joinpath(ds_dirname, ds_outpath.name))

# workaround: print cp command to use in shell
print(f"cp '{ds_outpath}' '{coclico_data_dir.joinpath(ds_dirname, ds_outpath.name)}'")

cp '/home/calkoen/ddata/tmp/cbca_CF.nc' '/p/11205479-coclico/data/06_adaptation_jrc/cbca_CF.nc'


In [55]:
pathlib.Path("/p/11205479-coclico/data/06_adaptation_jrc/cbca_CF.nc").exists()

True